In [94]:
import wikipedia
wikipedia.set_lang('ja')
# p = wikipedia.page('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)')
# p = wikipedia.page('ウレロ☆シリーズ')
p = wikipedia.page('アルコ&ピースのオールナイトニッポンシリーズ')

In [117]:
import re
import wikipedia
import bs4.element as bs4elem
from bs4 import BeautifulSoup

class WikiScraper(object):
    def __init__(self):
        self._page = ''
        self._bsObj = None
        self._headlines = []
        self._headline_tag = 'h3'
        self._last_key = ''
        self._result = dict()
    
    def __str__(self):
        output = ''
        for k, v in self._result.items():
            output +=  k + ' : ' 
            for staff in set(v):
                output += staff + ',       '
            output += u'\n'
        return output.encode('utf-8')

    
    def load_wiki(self, name, lang):
        wikipedia.set_lang(lang)
        try:
            self._page = wikipedia.page(name)
        except:
            print('Can not open wikipage of '+name+' with lang '+lang)

        self._bsObj = BeautifulSoup(self._page.html(), "html.parser")   
    
    
    def load_html(self, html):
        self._bsObj = BeautifulSoup(html, "html.parser")   
    
    
    def find_headline(self, name):
        headers = self._bsObj.find_all(class_='mw-headline')
        self._headlines = []
        for h in headers:
                if h.text == name:
                    self._headline_tag = h.parent.name
                    self._headlines.append(h)

    def cleanup_text(func):
        def wrapper(self, key, data):
            value = []
            data = data.encode('utf-8')
            data = re.sub('(（|\(|\（|［|\[).+?(）|\）|\)|］|\])','', data) #remove brackets
            data = data.strip()  #remove \n, and blank from head and tail

            #exception
            exception = [u'A・T・C事務所']
            for e in exception:
                if e.encode('utf-8') in data:
                    value.append(e)
                    data = data.replace(e.encode('utf-8'),'')
            
            value = func(key, data, value)
            
            if key in self._result:
                self._result[key].extend(value)
            else:
                 self._result[key] = value

        return wrapper

    @cleanup_text
    def cleanup_value_list(key, data, value):
        data = re.split('-', data)[0]
        value.extend(re.split(u'[,、・/／→\n]', data.decode('utf-8'))) #separate multiple staffs    
        for i, v in enumerate(value): #remove ※
            value[i]  = re.split(u'※', v)[0].strip()
        
        return value

    @cleanup_text
    def cleanup_value_infobox(key, data, value): 
        data = re.split(u'[※]', data)[0]
        value.extend(re.split(u'[,、・/→\n]', data.decode('utf-8'))) #separate multiple staffs    
        return value
                
    def find_list(self, elt, parent=''):
        for e in elt.find_all('ul'):
            for s in e.parent.strings:
                parent += s.strip() + ' '
                self._last_key = parent
                break            
            self.find_list(e, parent)
            parent = ''
            e.parent.decompose()
        for e in elt.find_all(['dd', 'li']):
            data = e.text
            if e.name == 'dd':
                data = self._last_value = self._result[self._last_key][-1]+ data
                key = self._last_key
                self._last_value = ''
            
            elif not re.search(u'[：]',data): #e.name == 'dd':
                key = parent.strip()
                self._last_value = ''
                
            else:       
                datas = re.split(u'[：-]', data)
                data = datas[1]
                key = parent + datas[0].strip()

            value = self.cleanup_value_list(key, data)

            self._last_key = key
                
    def get_list_from_headline(self, name):
        self.find_headline(name)
        for headline_start in self._headlines:
            finish = False
            for elt in headline_start.parent.nextSiblingGenerator():
#                 for e in elt:
#                     if isinstance(e, bs4elem.Tag) and e.get('class') == [u'mw-headline']:
#                         finish = True
#                         break
#                 if finish:
#                     break
#                 print elt.name, self._headline_tag
#                 print elt
                if elt.name is not None and re.match('h[1-6]', elt.name) and elt.name <= self._headline_tag:
                    break

                if hasattr(elt, 'text'):
                    self.find_list(elt)

                
    def get_table(self, class_name='infobox'):
        tables = self._bsObj.findAll('table',{'class':class_name})
        for table in tables:
            rows = table.findAll('tr')
            for row in rows:
                index = row.find('th')
                if index is not None:
                    key = index.get_text()
                    value = row.find('td')
                    if value is not None:
                        value_str = re.sub('</*br/*>', '\n', str(value))
                        value = BeautifulSoup(value_str, "html.parser")
    #                    value = value.get_text().split('\n')
                        value = self.cleanup_value_infobox(key, value.get_text())
    #                     self._result[key] = value

                    
wsc = WikiScraper()
#wsc.load_wiki('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)', 'ja')
wsc.load_html(p.html())
wsc.get_list_from_headline(u'スタッフ')
wsc.get_table('infobox')
print wsc

プロデューサー : 宗岡芳樹,       節丸雅予,       
AD : サイトー,       
ディレクター : 宗岡芳樹,       石井玄,       
パーソナリティ : アルコ&ピース,       
放送時間 : 毎週木曜日 27:00 - 28:30,       
放送局 : ニッポン放送,       
構成 : 畠山健,       福田卓也,       
テーマ曲 : 下記「テーマ曲」参照,       
ネットワーク : NRN,       
ジャンル : バラエティ番組,       
放送方式 : 生放送,       
放送期間 : 2012年8月31日,       12月1日,       2013年1月5日,       4月4日 - 2016年3月24日,       ,       
公式サイト : 公式サイト,       

